In [1]:
!pip install emot
!pip install demoji
!pip install language_tool_python
!pip install num2words

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import nltk,ast,re,string,time,math,num2words,gensim,demoji,language_tool_python,unicodedata
from dateutil import parser
from bs4 import BeautifulSoup
from collections import Counter 
from emot.emo_unicode import EMOTICONS_EMO
from math import sqrt

from google.colab import drive
drive.mount('/content/drive')

#punkt for tokenization
#nltk.download('punkt')

#load word vectors
#model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/WordVectors/GoogleNews-vectors-negative300.bin', binary=True)
#model.init_sims(replace=True)

#import language check
#tool = language_tool_python.LanguageTool('en-US')

#string.printable returns all sets of punctuation, digits, ascii_letters and whitespace
printable = set(string.printable)

     |████████████████████████████████| 61 kB 11 kB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 101 kB 7.2 MB/s 
Mounted at /content/drive


In [3]:
#decorator function for calculating the total time reqired to execute various function
def calc_time(func):
  def inner(*args, **kwargs):
    st = time.time()
    result = func(*args,**kwargs)
    end = time.time()-st
    print("Total time required: {:.3f} ms".format(end * 1000))
    return result
  return inner

#function to split dataframe into chuncks
def split_dataframe(df, chunk_size = 2000):
  chunks = list()
  num_chunks = len(df) // chunk_size + 1
  for i in range(num_chunks):
      chunks.append(df[i*chunk_size:(i+1)*chunk_size])
  return chunks

In [9]:
#function reads data collected and divides into chunks for data preprocessing
#this is a one time function call to divide into chunks
@calc_time
def divide_chunks():

  #read data 
  df = pd.read_csv("/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/Chunks/cleanchunk103.csv", header=None, names=["Content","Label"])

  #drop duplicates
  df = df.drop_duplicates(subset = ["Content"])
  df.reset_index(drop = True, inplace=True)

  #remove nan values from the dataframe
  df = df[df['Content'].notna()]

  #rest index
  df.reset_index(drop = True, inplace=True)

  #split dataframe into chuncks
  sp_df = split_dataframe(df)

  #store the chuncks into a seprate csv file
  for i in range(0,len(sp_df)):
    sp_df[i].to_csv("/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/Chunks/cleanchunk103{}.csv".format(i),index=False)

#divide_chunks()

Total time required: 1044.823 ms


In [10]:
#function to remove date time values from the text
def is_valid_date(date_str):
  try:
    parser.parse(date_str, fuzzy_with_tokens=True)
    return True
  except:
    return False

#function to simplify text and remove accented numbers
def simplify(text):
	import unicodedata
	try:
		text = unicode(text, 'utf-8')
	except NameError:
		pass
	text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
	return str(text)

#function to convert numbers to words
def convert_num_to_words(utterance):
  utterance = ' '.join([num2words.num2words(i) if i.isdigit() else i for i in utterance.split()])
  
  return utterance

#fucntion to strip accented characters
def strip_accents(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

#function to convert word to vector
def word2vec(word):
  # count the characters in word
  cw = Counter(word)
  # precomputes a set of the different characters
  sw = set(cw)
  # precomputes the "length" of the word vector
  lw = sqrt(sum(c*c for c in cw.values()))

  # return a tuple
  return cw, sw, lw

#function to calculate cosine distance
def cosdis(v1, v2):
  # which characters are common to the two words?
  try:
    common = v1[1].intersection(v2[1])
    # by definition of cosine distance we have
    return sum(v1[0][ch]*v2[0][ch] for ch in common)/v1[2]/v2[2]
  except IndexError:
    return 0

#function for Edit Distance Similirty
def edit_distance(str1, str2):
  m, n = len(str1), len(str2)
  if m==0 and n!=0:
    return n, 1-n/max(m, n)
  elif m!=0 and n==0:
    return m, 1-m/max(m, n)
  elif m==0 and n==0:
    try:
      1-0/0
    except ZeroDivisionError as z:
        print("string can not be null",z)
    return math.nan, math.nan
  else:
    d = np.zeros((n+1, m+1))
    d[0] = np.arange(m+1)
    d[:, 0] = np.arange(n+1)
    for i in range(1, n+1):
      for j in range(1, m+1):
        if str1[j-1]==str2[i-1]:
          temp = 0
        else:
          temp = 1
          d[i, j] = min(d[i-1, j]+1, d[i, j-1]+1, d[i-1, j-1]+temp)
    return d[n, m], 1-d[n, m]/max(m, n)

In [11]:
#data preprocessing pipeline 
def data_preprocessing(text,sc_contractions,dc_contractions,cnt):
  
  print("data preprocessing going on for row ",cnt)
  #convert words to lower case
  text = text.lower()
  #print("\n step1 \n",text)

  #remove html tags 
  text = BeautifulSoup(text, "html").text
  #print("\n step2 \n",text)

  #remove @usermentions and url       
  text = re.sub(r"(?:\@|https?\://)\S+", "", text) 
  #print("\n step3 \n",text)

  #convert emojis to text 
  text = demoji.replace_with_desc(text)
  #print("\n step4 \n",text)

  #convert emoticons to text
  text = text.split(' ')
  for i in range(0,len(text)):
    if text[i] in EMOTICONS_EMO:
      text[i]=EMOTICONS_EMO[text[i]]
    else:
      pass

  #combine list into a sentence
  text = ' '.join(text)
  #print("\n step5 \n",text)

  #remove https string
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE).replace(":","")
  #print("\n step6 \n",text)

  #remove new line spaces 
  text = re.sub(r'<br />', ' ', text) 
  #print("\n step7 \n",text)

  #remove hyperlink
  text = re.sub(r'\<a href', ' ', text) 
  #print("\n step8 \n",text)

  #replace single contractions with their longer forms 
  text = text.split(' ')
  new_text = []
  for word in text:
    if word in sc_contractions:
      new_text.append(sc_contractions[word])
    else:
      new_text.append(word)

  #combine the expanded text together    
  text = " ".join(new_text)
  #print("\n step9 \n",text)

  #create a tempory variable to copy text output
  temp_text = text

  #expand the double contractions
  text = text.split(' ')
  new = []
  c=0
  for k,v in dc_contractions.items():
    if k in text:
      if c==0:
        #print(k)
        for i in v.split('/'):
          #print(i)
          new.append(temp_text.replace(k,i))
        c=1
      else:
        m = len(new)
        n = len(v.split("/"))
        for p in range(0,len(new)):
          for f in v.split('/'):
            new.append(new[p].replace(k,f))  
        new = new[-(m*n):] 

  text = ' '.join(text)
  wd = []
  for i in range(0,len(new)):
    wd.append(model.wmdistance(text.lower().split(' '),new[i].lower().split(' ')))
  #print(wd)
  #correct the grammer of the sentence if its incorrect it corrects the grammatical errors
  if len(wd)==0:
    text = tool.correct(text)
  else:
    text = tool.correct(new[wd.index(min(wd))])

  #print("\n step10 \n",text)

  #remove date time values from the text
  text = [text]
  text = [' '.join([w for w in line.split() if not is_valid_date(w)]) for line in text]
  text = ' '.join(text)
  #print("\n step11 \n",text)

  #remove accented numbers and simplify the text
  text = simplify(text)
  #print("\n step12 \n",text)

  #convert number to words
  text =  convert_num_to_words(text)
  #print("\n step13 \n",text)
      
  #remove words that start with &
  text = " ".join(filter(lambda x:x[0]!='&', text.split())) 
  #print("\n step14 \n",text)

  #strip accented characters
  text = strip_accents(text)
  #print("\n step 15 \n",text)

  #remove unwanted charecters like puntuation and others
  text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text) 
  #print("\n step16 \n",text)

  #remove digits this is for exceptional cases
  text = ''.join([i for i in text if not i.isdigit()])
  #print("\n step17 \n",text)

  #remove slashes
  text = re.sub(r'\'', ' ', text)
  #print("\n step18 \n",text)

  #remove trailing spaces   
  text = " ".join(text.split())
  #print("\n step19 \n",text)

  #filter to remove punctuations,digits, ascii_letters and whitespaces
  text = "".join(list(filter(lambda x: x in printable, text))) 
  #print("\n step20 \n",text)

  #keep only words in the sentence 
  text = text.split()
  text = ' '.join(x for x in text if x.isalpha())
  #print("\n step21 \n",text)

  return text

In [13]:
#Program to measure the similarity between
#two sentences using cosine similarity.
#function to correct words
def ForDataSet(data_path = '/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/Chunks/cleanchunk960.csv',english_path='/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/Contractions/english.csv'):
  data = pd.read_csv(data_path)
  #convert to lower text
  data["Content"] = data["Content"].apply(lambda x: x.lower())
  c = iter(range(len(data)))
  #read badwords file
  words = pd.read_csv(english_path, header=None, names=["word_replace"])
  words = list(words["word_replace"])
  #convert word to vec
  text2 = [word2vec(w) for w in words]
  for i,j in data.iterrows():
    print("processing text for row",next(c))
    for word in data["Content"][i].split(' '):
      #find cosine similarity
      cs = [cosdis(word2vec(str(word)),w) for w in text2]
      if max(cs) > 0.6:
        data["Content"][i].replace(word,words[cs.index(max(cs))])
      #find edit distance
      ed = [edit_distance(str(word),w) for w in words]
      if max(ed)[1] > 0.5:
        data["Content"][i].replace(word,words[ed.index(max(ed))])
    
  return data

In [14]:
#function to read all the data files and dictionary files
def read_data_files_preprocess():

  #start time 
  st=time.time()
  '''
  #read data 
  df = pd.read_csv("/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/Chunks/cleanchunkyx34.csv")

  #rest index
  df.reset_index(drop = True, inplace=True)

  print("\n length of the dataset is ",len(df))
  print("\n count of labels",df["Label"].value_counts())
  
  #import the dictionary of single contractions 
  with open('/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/Contractions/Contractions','r') as f:
    data = f.read()
         
  #reconstructing the data as a dictionary
  sc_contractions = ast.literal_eval(data)

  #import the dictionary of double contractions 
  with open('/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/Contractions/DoubleContractions','r') as f:
    data = f.read()

  #reconstructing the data as a dictionary
  dc_contractions = ast.literal_eval(data)

  #apply preprocess function on the columns of the dataframe
  c = iter(range(len(df)))
  df['Content'] = df['Content'].apply(lambda x: data_preprocessing(x,sc_contractions,dc_contractions,next(c)))

  #write preprocesssed data to csv file
  df.to_csv("/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/DataPreprocessingCode/preprocessed_data_yidong_devansh.csv",mode='a', index=False, header=False)
  '''
  #call the function to correct words
  df = ForDataSet()

  #write preprocesssed data to csv file
  df.to_csv("/content/drive/MyDrive/COMP-5800-YDE-Yidong-Devansh-Final-Project/DataPreprocessingCode/final_preprocessed_data_yidong_devansh.csv",mode='a', index=False, header=False)
  
  print("total time to generate data and write in csv file ",time.time()-st)

In [15]:
if __name__:read_data_files_preprocess()

processing text for row 0
processing text for row 1
processing text for row 2
processing text for row 3
processing text for row 4
processing text for row 5
processing text for row 6
processing text for row 7
processing text for row 8
processing text for row 9
processing text for row 10
processing text for row 11
processing text for row 12
processing text for row 13
processing text for row 14
processing text for row 15
processing text for row 16
processing text for row 17
processing text for row 18
processing text for row 19
processing text for row 20
processing text for row 21
processing text for row 22
processing text for row 23
processing text for row 24
processing text for row 25
processing text for row 26
processing text for row 27
processing text for row 28
processing text for row 29
processing text for row 30
processing text for row 31
processing text for row 32
processing text for row 33
processing text for row 34
processing text for row 35
processing text for row 36
processing 